# Project 1
## 1.
Objective: $Min \sum_{t=1}^{T} f_ty_t + \sum_{t=1}^{T} c_tx_t + \sum_{t=1}^{T} h_ts_t$

Subject to:
$d_t = x_t + s_{t-1} - s_t \qquad \forall t \in H$
$x_t \leq D_ty_t \qquad \forall t \in H$

$y_t \in [0,1]$
<br>
$x_t, s_t \geq 0$

Where:
$D_t = \sum_{i=t}^{T} d_i$
$s_0, s_T = 0$

In [190]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

# Import data and define constants

T = 24  # Num periods
rep = 2  # Tab number

tabs = {'6-1': '6-periods (1)',
    '6-2': '6-periods (2)',
    '12-1': '12-periods (1)',
    '12-2': '12-periods (2)',
    '24-1': '24-periods (1)',
    '24-2': '24-periods (2)',
    '52-1': '52-periods (1)',
    '52-2': '52-periods (2)',
    '104-1': '104-periods (1)',
    '104-2': '104-periods (2)'}

xls = pd.ExcelFile(r'../ULSP-instancesR.xlsx')

df = pd.read_excel(xls, sheet_name=tabs[f'{T}-{rep}'])

# Add zero row and adjust index to align with indicies prompt
df.loc[-1] = [0]*6
df.index = df.index + 1
df = df.sort_index()

# Constants
d = df['Demand Forecast']
f = df['Setup Cost']
c = df['Production cost']
h = df['Holding cost']
b = df['Backlogging cost']

D = [d[i:].sum() for i in range(len(d))]

# Set
H = range(1,T+1)
H_0 = range(T+1)

# Gurobi environment
ENV = gp.Env()
ENV.setParam('OutputFlag', 0)


Restricted license - for non-production use only - expires 2025-11-24


In [191]:
def extract_model_vars(x, y, s):
    # ---- Extract variable values ----
    x_values = np.zeros(len(H_0))
    y_values = np.zeros(len(H_0))
    s_values = np.zeros(len(H_0))

    for t in H:
        x_values[t] = x[t].X
        y_values[t] = y[t].X
        s_values[t] = s[t].X

    return x_values, y_values, s_values


def inequality_separation(x, y, s):
    S = {}
    v = {}
    Djl = {}
    for l in H:
        S[l] = set()
        for j in range(1,l+1):
            Djl[(j,l)] = d[j:l+1].sum()
            if x[j] - Djl[(j,l)]*y[j] > 0:
                S[l].add(j)
        # v[l] = sum([x[j] - Djl[(j,l)]*y[j] for j in range(1,l+1)]) - s[l]
        v[l] = sum([x[j] - Djl[(j,l)]*y[j] if j in S[l] else 0]) - s[l] # TODO: confirm, error in problem 5, summation on 3rd line should be j \in S_l, not t \in S_l?
    return v, S, Djl


def run_model(relax=False, isp=False):
    start_time = time.time()

    # ---- Initiate model ----
    model = gp.Model(env=ENV)

    # ---- Decision variables ----

    # Units produced
    x = model.addVars(T+1, vtype=GRB.CONTINUOUS, name='x')

    # Whether a lot is made
    if relax:
        y = model.addVars(T+1, vtype=GRB.CONTINUOUS, name='y')
    else:
        y = model.addVars(T+1, vtype=GRB.BINARY, name='y')

    # Amount held in inventory at end of t
    s = model.addVars(T+1, vtype=GRB.CONTINUOUS, name='s')


    # ---- Constraints ----

    # Initial values
    model.addConstr(s[0] == 0)
    model.addConstr(x[0] == 0)
    model.addConstr(y[0] == 0)

    # Material balance
    model.addConstrs(int(d[t]) == x[t] + s[t-1] - s[t] for t in H)

    # y definition
    model.addConstrs(x[t] <= D[t]*y[t] for t in H)


    # ---- Objective ----
    obj = gp.LinExpr()

    for t in H:
        obj.addTerms([f[t], c[t], h[t]], [y[t], x[t], s[t]])

    model.setObjective(obj, GRB.MINIMIZE)

    # ---- Optimize ----
    model.optimize()

    # ---- Extract values ----
    x_values, y_values, s_values = extract_model_vars(x, y, s)

    if isp:
        if not relax:
            print("WARNING: isp is only designed to be used when relax=True")

        v, S, Djl = inequality_separation(x_values, y_values, s_values)

        count = 0
        while max(v.values())>0 and count <= 100:
            # l_max = max(v.items(), key=lambda x: x[1])[0]
            # model.addConstrs(gp.quicksum(x[t] for t in S[l]) <= gp.quicksum(Djl[(t,l)]*y[t] for t in S[l]) + s[l]for l in range(1,l_max+1))
            print("v: ", v)
            l = max(v.items(), key=lambda x: x[1])[0]
            print("l: ", l)
            print("v_max: ", max(v.items(), key=lambda x: x[1]))
            model.addConstr(gp.quicksum(x[t] for t in S[l]) <= gp.quicksum(Djl[(t,l)]*y[t] for t in S[l]) + s[l])
            model.optimize()
            x_values, y_values, s_values = extract_model_vars(x, y, s)

            v, S, Djl = inequality_separation(x_values, y_values, s_values)

            print("final v: ", v)

            if count == 99:
                print("WARNING: isp max iterations exceeded")
            count += 1

    end_time = time.time()
    total_time = end_time - start_time

    return x_values, y_values, s_values, total_time

def print_model_results(relax, isp):
    x_values, y_values, s_values, total_time = run_model(relax=relax, isp=isp)
    print("x: ", x_values)
    print("y: ", y_values)
    print("s: ", s_values)
    print("total_time: ", total_time)

print("MILP model without relaxation:")
print_model_results(relax=False, isp=False)

print()
print("LP model with relaxation:")
print_model_results(relax=True, isp=False)

MILP model without relaxation:
x:  [   0. 1067.    0. 1366.    0.  853.    0.  695.    0.    0.    0.  842.
    0. 1245.    0.    0.  958.    0.  585. 1876.    0.    0.    0. 1157.
    0.]
y:  [0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1.
 0.]
s:  [   0.  718.    0.  436.    0.    0.    0.  363.  363.    0.    0.  414.
    0.  842.  199.    0.  628.    0.    0. 1036.  785.  371.    0.  512.
    0.]
total_time:  0.009994983673095703

LP model with relaxation:
x:  [   0.  349.  718. 1366.    0.  853.    0.  332.    0.  363.    0.  428.
  414.  403.  643.  199.  330.  628.  585.  840.  665.    0.  371.  645.
  512.]
y:  [0.         0.03278843 0.06974259 0.14263339 0.         0.10388503
 0.         0.04512096 0.         0.05166524 0.         0.06423533
 0.06639936 0.06923209 0.11867848 0.04167539 0.07211538 0.14790391
 0.16169154 0.27695351 0.30323757 0.         0.24280105 0.55747623
 1.        ]
s:  [  0.   0.   0. 436.   0.   0.   0.   0.   0.   0.   0.   0.   

# 2.
Computation times are comparable at around 0.001s for the 6-element dataset and 0.006 for the 104-element dataset. Solutions for production quantities and holding are the same for the 6-element set, but not the 104-element set, although the y variable indicating whether a lot is produced is no longer strictly 0 or 1. If it is 0, no lot is produced, but when a lot is produced, y is a fractional value. This will not affect the production plan, but it will give an artificially lower objective.

## TODO: make code that automatically compares all variables for all datasets and gives a summary output?

# 3.
 Yes, we could use a heuristic here (i.e. dynamic programming) to get the same result as the MILP. In fact, this would be a good application for the Silver-Meal heuristic, which is similar to (Wagner-Whitin, 1958), except the former calculates cost per unit produced, and (Wagner-Whitin, 1958) looks at the absolute cost to fill the demand in a particular period.

## TODO: The question asks to identify a particular pattern?

In [192]:
# See https://www.youtube.com/watch?v=7vE-gm9qxpk

z = np.zeros(T+1)
x_ww = np.zeros(T+1)

for t in H:
    t_t = t
    z_t = np.zeros(t+1)
    while t_t > 0:

        z_t[t_t] = f[t_t]
        t_t -= 1

    z[t] = min(z_t)

# TODO: finish # 3

# 4.
This inequality states that the amount of units produced in any subset of periods must be less than or equal to the demand of those periods plus the ending inventory.
Indeed, the constraint for the definition of $y_t$ is:
$x_t \leq D_ty_t \qquad \forall t \in H$

If this is the case, then cumulative (i.e. summation of) $x_t$ must also be less than the cumulative $Dy_t$. $s_l$ is defined to be nonnegative, so adding this to the right side does not change the inequality.

# 5.

See inequality_separation() function in cell 3. Example output is shown below.

In [193]:
x, y, s, _ = run_model(relax=True)
v, S, Djl = inequality_separation(x, y, s)


print(v)
print(max(v.items(), key=lambda x: x[1]))


{1: 337.5568395340098, 2: 667.9248178727538, 3: 797.3509449723294, 4: 0.0, 5: 764.3860674704665, 6: 0.0, 7: 317.01984234846424, 8: 0.0, 9: 344.2455166524338, 10: 0.0, 11: 400.5072790034519, 12: 386.51066559743384, 13: 375.0994674454561, 14: 566.6897379106681, 15: 190.70659685863873, 16: 306.2019230769231, 17: 535.1163447951012, 18: 490.410447761194, 19: 607.3590504451039, 20: 174.88736890104883, 21: 0.0, 22: 280.9208115183246, 23: 285.4278305963699, 24: 0.0}
(3, 797.3509449723294)


# 6.

# TODO: Something is wrong, not all y's are binary in all cases

In [195]:
print_model_results(relax=True, isp=True)



v:  {1: 337.5568395340098, 2: 667.9248178727538, 3: 797.3509449723294, 4: 0.0, 5: 764.3860674704665, 6: 0.0, 7: 317.01984234846424, 8: 0.0, 9: 344.2455166524338, 10: 0.0, 11: 400.5072790034519, 12: 386.51066559743384, 13: 375.0994674454561, 14: 566.6897379106681, 15: 190.70659685863873, 16: 306.2019230769231, 17: 535.1163447951012, 18: 490.410447761194, 19: 607.3590504451039, 20: 174.88736890104883, 21: 0.0, 22: 280.9208115183246, 23: 285.4278305963699, 24: 0.0}
l:  3
v_max:  (3, 797.3509449723294)
final v:  {1: 337.5568395340098, 2: 667.9248178727538, 3: -436.0, 4: 0.0, 5: 764.3860674704665, 6: 0.0, 7: 317.01984234846424, 8: 0.0, 9: 344.2455166524338, 10: 0.0, 11: 400.5072790034519, 12: 386.51066559743384, 13: 375.0994674454561, 14: 566.6897379106681, 15: 190.70659685863873, 16: 306.2019230769231, 17: 535.1163447951012, 18: 490.410447761194, 19: 607.3590504451039, 20: 174.88736890104883, 21: 0.0, 22: 280.9208115183246, 23: 285.4278305963699, 24: 0.0}
v:  {1: 337.5568395340098, 2: 667.

# 7.
New formulation:

$Min \sum_{t=1}^{T} f_ty_t + \sum_{t=1}^{T}\sum_{q=1}^{t} c_td_tw_{q,t} + \sum_{t=1}^{T}(h_t\sum_{i=1}^{t-1}d_i\sum_{q=1}^{i}w_{q,i} - \sum_{j=1}^{t-1}d_j)$
<br>
(Setup cost + unit production costs + (accumulated inventory to period t-1 - demand that has been met up to period t-1))

Subject to:
$\sum_{q=1}^{t}w_{q,t} = 1 \qquad  \forall t \in T$     (Ensures demand is met)

$\sum_{t=q}^{T}w_{q,t} \leq y_q \qquad  \forall q \in T$       (Defines y as 1 if any products are produced, 0 if not)
<br>
$y_t \in [0,1]$
<br>
$0 \leq w_{q,t} \leq 1$



In [208]:
def fplr_model(relax=False):

    start_time = time.time()

    # ---- Initiate model ----
    model = gp.Model(env=ENV)

    # ---- Decision variables ----

    # Units produced
    w = model.addVars(T+1, T+1, vtype=GRB.CONTINUOUS, name='w')

    # Whether a lot is made
    if relax:
        y = model.addVars(T+1, vtype=GRB.CONTINUOUS, name='y', ub=1)
    else:
        y = model.addVars(T+1, vtype=GRB.BINARY, name='y')


    # ---- Constraints ----

    # Ensure demand is met
    model.addConstrs(gp.quicksum(w[q,t] for q in range(1,t+1)) == 1 for t in H)

    # Define setup variable y
    model.addConstrs(gp.quicksum(w[q,t] for t in range(q,T+1)) <= y[q] for q in H)


    # ---- Objective ----
    obj = gp.LinExpr()

    for t in H:
        obj.addTerms(f[t], y[t])
        for q in range(1,t+1):
            obj.addTerms(c[t]*d[t], w[q,t])
        for i in range (1,t):
            for q in range(1,i+1):
                obj.addTerms(h[t]*d[i], w[q,i])
        # Below is not needed, as it is a constant
        # for j in range(1,t):
        #     obj.addTerms(-h[t]*d[j])

    model.setObjective(obj, GRB.MINIMIZE)

    # ---- Optimize ----
    model.optimize()

    # ---- Extract variable values ----
    w_values = np.zeros((len(H_0),len(H_0)))
    y_values = np.zeros(len(H_0))

    for q in H:
        for t in H:
            w_values[q,t] = w[q,t].X
        y_values[q] = y[q].X

    end_time = time.time()
    total_time = end_time - start_time

    return w_values, y_values, total_time

w_values, y_values, total_time = fplr_model(relax=True)

units_per_period = np.zeros(len(H_0))

# TODO: need to use d[t] to calculate units made per period
for i in range(len(units_per_period)):
    units_per_period[i] = np.sum(w_values[:,i])

print("w: ", w_values)
print("y: ", y_values)
print("total_time: ", total_time)
print("units_per_period: ", units_per_period)


w:  [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0